- https://github.com/ukairia777/tensorflow-nlp-tutorial/blob/main/22.%20Fine-tuning%20GPT-2%20(Cls%2C%20Chatbot%2C%20NLI)/22-4.%20kogpt2_nsmc_tpu.ipynb

In [ ]:
pip install transformers

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import transformers
transformers.__version__

'4.35.2'

In [3]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from transformers import AutoTokenizer, TFGPT2Model
from tensorflow.keras.preprocessing.sequence import pad_sequences

## 데이터 불러오기

In [43]:
train_data = pd.read_csv('/content/drive/MyDrive/CUAI 컨퍼런스/data/merged_df(감정 분류).csv')[['content', 'label']]
test_data = pd.read_csv('/content/drive/MyDrive/CUAI 컨퍼런스/data/test_data.csv')
test_data['content'] = test_data['title'] + ' ' + test_data['content']

In [44]:
print('훈련용 리뷰 개수 :',len(train_data)) # 훈련용 데이터 개수 출력

훈련용 리뷰 개수 : 45999


In [45]:
print('테스트용 리뷰 개수 :',len(test_data)) # 테스트용 데이터 개수 출력

테스트용 리뷰 개수 : 390


In [46]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
train_data = train_data.reset_index(drop=True)
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [47]:
test_data = test_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
test_data = test_data.reset_index(drop=True)
print(test_data.isnull().values.any()) # Null 값이 존재하는지 확인

False


## 전처리

#### 불용어 처리

In [48]:
# # 문자열 아닌 데이터 모두 제거
train_data['content'] = [content for content in train_data['content'] if type(content) is str]
test_data['content'] = [content for content in test_data['content'] if type(content) is str]

In [49]:
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']

In [50]:
# stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을','것','이다','게','에서','거','로','수','에게','요']

In [51]:
pip install konlpy

In [52]:
from konlpy.tag import Okt
from tqdm import tqdm

# KoNLPy의 Okt 형태소 분석기를 사용하여 불용어 제거
okt = Okt()

def remove_stopwords(text):
    words = okt.pos(text, stem=False)  # 형태소 분석, (stem=True -> 기본형?으로 나옴)
    filtered_words = [word for word, pos in words if word not in stopwords]  # 불용어 제거
    return ' '.join(filtered_words)


# 'content' 열에 있는 각 텍스트에 대해 불용어 제거 수행
tqdm.pandas()
train_data['content'] = train_data['content'].progress_apply(remove_stopwords)
test_data['content'] = test_data['content'].progress_apply(remove_stopwords)

100%|██████████| 390/390 [00:04<00:00, 81.71it/s] 


#### 텍스트 정규화

In [53]:
import re
from tqdm import tqdm

def text_normalization(text):
    # 소문자 변환
    text = text.lower()

    # 숫자 제거
    text = re.sub(r'\d+', '', text)

    # 특수문자 제거
    # text = re.sub(r'[^\w\s]', '', text)

    return text

tqdm.pandas()
train_data['content'] = train_data['content'].progress_apply(text_normalization)
test_data['content'] = test_data['content'].progress_apply(text_normalization)

100%|██████████| 390/390 [00:00<00:00, 84235.98it/s]


#### 반복 표현 제거(ex. ㅋㅋㅋㅋㅋㅋ,ㅎㅎㅎㅎㅎ)

In [54]:
def replace_repeated_chars(text):
    # 두 번 이상 반복되는 글자(예: ㅋㅋㅋ, ㅎㅎㅎ)를 찾아 해당 글자의 두 번 반복으로 치환
    pattern = re.compile(r'(.)\1{1,}', re.DOTALL)

    # 정규식에 맞게 두 번 이상 반복되는 글자를 찾아 치환
    text = pattern.sub(r'\1\1', text)

    return text

# 'content' 열에 있는 각 텍스트에 대해 두 번 이상 반복되는 글자를 찾아 치환
tqdm.pandas()
train_data['content'] = train_data['content'].progress_apply(replace_repeated_chars)
test_data['content'] = test_data['content'].progress_apply(replace_repeated_chars)

100%|██████████| 390/390 [00:00<00:00, 36051.81it/s]


In [55]:
df_length = train_data['content'].astype(str).apply(len)
print('글 길이 최댓값: {}'.format(np.max(df_length)))
print('글 길이 최솟값: {}'.format(np.min(df_length)))
print('글 길이 평균값: {:.2f}'.format(np.mean(df_length)))
print('글 길이 중간값: {}'.format(np.median(df_length)))
print('글 길이 제1사분위: {}'.format(np.percentile(df_length,25)))
print('글 길이 제3사분위: {}'.format(np.percentile(df_length,75)))

글 길이 최댓값: 133
글 길이 최솟값: 1
글 길이 평균값: 30.01
글 길이 중간값: 29.0
글 길이 제1사분위: 19.0
글 길이 제3사분위: 39.0


In [56]:
train_data = train_data[train_data['content'].str.len() > 5]
train_data.shape

(45853, 2)

In [57]:
train_data.drop_duplicates(subset=['content'], inplace=True, ignore_index=True) # 중복 제거

## 토큰화

In [58]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', pad_token='<pad>')

In [59]:
print(tokenizer.tokenize("전율을 일으키는 영화. 다시 보고싶은 영화"))

['▁전', '율을', '▁일으키는', '▁영화', '.', '▁다시', '▁보고', '싶', '은', '▁영화']


In [60]:
print(tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화"))

[9034, 13555, 16447, 10584, 389, 9427, 10056, 7898, 8135, 10584]


In [61]:
print(tokenizer.decode(3))

<pad>


In [62]:
max_seq_len = 128

In [63]:
encoded_result = tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화", max_length=max_seq_len, pad_to_max_length=True)
print(encoded_result)
print('길이 :', len(encoded_result))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[9034, 13555, 16447, 10584, 389, 9427, 10056, 7898, 8135, 10584, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
길이 : 128


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [64]:
answer = '안녕하세요 반가워요'

# BOS와 EOS 토큰을 추가하여 리스트로 만듭니다.
answers = [tokenizer.bos_token] + [answer] + [tokenizer.eos_token]

# 리스트를 문자열로 변환합니다.
answers_str = ' '.join(answers)

# 토크나이저를 사용하여 패딩과 트러케이션을 적용합니다.
encoded_inputs = tokenizer(answers_str, return_tensors="pt", max_length=256, padding=True, truncation=True)

print(encoded_inputs)

{'input_ids': tensor([[    1, 25906,  8702,  7801,  8084, 36230,  8102,  8084,   739,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


### 토큰화

In [65]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):

    input_ids, data_labels = [], []

    for example, label in tqdm(zip(examples, labels), total=len(examples)):

        bos_token = [tokenizer.bos_token]
        eos_token = [tokenizer.eos_token]
        tokens = bos_token + tokenizer.tokenize(example) + eos_token   # 토큰으로 나누고 bos, eos 토큰을 앞뒤로 붙여줌
        input_id = tokenizer.convert_tokens_to_ids(tokens)   # 토큰을 인덱스로 변경
        input_id = pad_sequences([input_id], maxlen=max_seq_len, value=tokenizer.pad_token_id, padding='post')[0]    # 패딩

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        input_ids.append(input_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    data_labels = np.asarray(data_labels, dtype=np.int32)

    return input_ids, data_labels

In [66]:
train_X, train_y = convert_examples_to_features(train_data['content'], train_data['label'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 45710/45710 [00:14<00:00, 3257.02it/s]


In [67]:
test_X, test_y = convert_examples_to_features(test_data['content'], test_data['label'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 390/390 [00:00<00:00, 3111.10it/s]


In [68]:
# 최대 길이: 128
input_id = train_X[0]
label = train_y[0]

print('단어에 대한 정수 인코딩 :',input_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))
print('레이블 :',label)

단어에 대한 정수 인코딩 : [    1 11275  8270 10607  9063 10056  9050  8185  7957 22990 36510     1
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3]
각 인코딩의 길이 : 128
정수 인코딩 복원 : </s> 옆집 아이 나 보고 아저씨 래.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

## 학습

In [69]:
model = TFGPT2Model.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.10.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'lm_head.weight']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [70]:
max_seq_len = 128

In [71]:
input_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
outputs = model([input_ids_layer])

In [72]:
print(outputs)

TFBaseModelOutputWithPastAndCrossAttentions(last_hidden_state=<KerasTensor: shape=(None, 128, 768) dtype=float32 (created by layer 'tfgpt2_model_2')>, past_key_values=(<KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model_2')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model_2')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model_2')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model_2')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model_2')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model_2')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model_2')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model_2')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by lay

In [73]:
class TFGPT2ForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name):
        super(TFGPT2ForSequenceClassification, self).__init__()
        self.gpt = TFGPT2Model.from_pretrained(model_name, from_pt=True)
        self.dropout = tf.keras.layers.Dropout(0.2)
        self.classifier = tf.keras.layers.Dense(1,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='sigmoid',
                                                name='classifier')

    def call(self, inputs):
        outputs = self.gpt(input_ids=inputs)
        cls_token = outputs[0][:, -1]
        cls_token = self.dropout(cls_token)
        prediction = self.classifier(cls_token)

        return prediction

TPU 사용법 : https://wikidocs.net/119990

In [74]:
# TPU 작동을 위한 코드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

In [75]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [76]:
with strategy.scope():
  model = TFGPT2ForSequenceClassification("skt/kogpt2-base-v2")
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  loss = tf.keras.losses.BinaryCrossentropy()
  model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.10.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'lm_head.weight']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [77]:
model.fit(train_X, train_y, epochs=2, batch_size=32, validation_split=0.2)

Epoch 1/2
1143/1143 [==============================] - 215s 124ms/step - loss: 0.3429 - accuracy: 0.8610 - val_loss: 0.3161 - val_accuracy: 0.8845
Epoch 2/2
1143/1143 [==============================] - 96s 84ms/step - loss: 0.2160 - accuracy: 0.9192 - val_loss: 0.1290 - val_accuracy: 0.9552


## 평가, 예측

In [78]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
# 모델 예측
predictions = model.predict(test_X)

# 이진 분류에서는 predictions를 0 또는 1로 변환해주어야 합니다.
binary_predictions = (predictions > 0.5).astype(int)

# 정밀도, 재현율, F1 스코어 계산
accuracy = accuracy_score(test_y, binary_predictions)
precision = precision_score(test_y, binary_predictions)
recall = recall_score(test_y, binary_predictions)
f1 = f1_score(test_y, binary_predictions)

print("accuracy: {:.4f}".format(accuracy))
print("precision: {:.4f}".format(precision))
print("recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

13/13 [==============================] - 11s 553ms/step
accuracy: 0.7769
precision: 0.9365
recall: 0.5990
F1 Score: 0.7307


In [ ]:
def sentiment_predict(new_sentence):

  bos_token = [tokenizer.bos_token]
  eos_token = [tokenizer.eos_token]
  tokens = bos_token + tokenizer.tokenize(new_sentence) + eos_token
  input_id = tokenizer.convert_tokens_to_ids(tokens)
  input_id = pad_sequences([input_id], maxlen=max_seq_len, value=tokenizer.pad_token_id, padding='post')[0]
  input_id = np.array([input_id])
  score = model.predict(input_id)[0][0]

  if(score > 0.5):
    print("{:.2f}% 확률로 우울글입니다.\n".format(score * 100))
  else:
    print("{:.2f}% 확률로 우울글이 아닙니다.\n".format((1 - score) * 100))

In [ ]:
sentiment_predict('보던거라 계속보고있는데 전개도 느리고 주인공인 은희는 한두컷 나오면서 소극적인모습에')

1/1 [==============================] - 7s 7s/step
93.12% 확률로 우울글이 아닙니다.



In [ ]:
sentiment_predict("스토리는 확실히 실망이였지만 배우들 연기력이 대박이였다 특히 이제훈 연기 정말 ... 이 배우들로 이렇게밖에 만들지 못한 영화는 아쉽지만 배우들 연기력과 사운드는 정말 빛났던 영화. 기대하고 극장에서 보면 많이 실망했겠지만 평점보고 기대없이 집에서 편하게 보면 괜찮아요. 이제훈님 연기력은 최고인 것 같습니다")

1/1 [==============================] - 1s 721ms/step
79.70% 확률로 우울글입니다.



In [ ]:
sentiment_predict("남친이 이 영화를 보고 헤어지자고한 영화. 자유롭게 살고 싶다고 한다. 내가 무슨 나비를 잡은 덫마냥 나에겐 다시 보고싶지 않은 영화.")

1/1 [==============================] - 1s 660ms/step
80.35% 확률로 우울글입니다.

